# Technical Factor - Directional Movement Index + Average Directional Index (DMIADX)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        m_ta_ema(IF(high - m_lag(high, 1) > 0, high - m_lag(high, 1), 0), 14) AS PDM,
        m_ta_ema(IF(m_lag(low, 1) - low > 0, m_lag(low, 1) - low, 0), 14) AS NDM,
        m_ta_ema(GREATEST(ABS(high - low), ABS(high - m_lag(close, 1)), ABS(low - m_lag(close, 1))), 14) AS TR,
        PDM / TR * 100 AS PDI,
        NDM / TR * 100 AS MDI,
        m_avg(ABS(PDI - MDI) / (PDI + MDI), 6) AS ADX,
        (ADX + m_lag(ADX, 6)) / 2 AS ADXR,

    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(PDI > MDI AND m_lag(PDI, 1) <= m_lag(MDI, 1), 1, 0) AS TRBY1,
        IF(PDI < MDI AND m_lag(PDI, 1) >= m_lag(MDI, 1), 1, 0) AS TRSL1,
        IF(ADX > 25 AND m_lag(ADX, 1) <= 25, 1, 0) AS TRBY2,
        IF(ADX < 20 AND m_lag(ADX, 1) >= 20, 1, 0) AS TRSL2,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(ADX > m_lag(ADX, 1), 1, 0) AS MTBY1,
        IF(ADX < m_lag(ADX, 1), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(ADX > 40 AND (ADX - m_lag(ADX, 1)) < 0, 1, 0) AS RVBY1,
        IF(ADX < 15 AND (ADX - m_lag(ADX, 1)) > 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(ADX, 1) <= 20 AND ADX > 20, 1, 0) AS BKBY1,
        IF(m_lag(ADX, 1) >= 25 AND ADX < 25, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        PDI,
        MDI,
        ADX,
        ADXR,
        TRBY1,
        TRSL1,
        TRBY2,
        TRSL2,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,PDI,MDI,ADX,ADXR,TRBY1,TRSL1,TRBY2,TRSL2,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-13,600519.SH,28.353567,19.324615,0.271339,0.290294,1,0,0,0,0,1,0,0,0,0
1,2025-02-14,600519.SH,26.560433,17.504213,0.214371,0.303767,0,0,0,0,0,1,0,0,0,0
2,2025-02-17,600519.SH,31.830542,15.072140,0.239654,0.310048,0,0,0,0,1,0,0,1,0,0
3,2025-02-18,600519.SH,27.026771,15.248496,0.251822,0.309827,0,0,0,0,1,0,0,1,0,0
4,2025-02-19,600519.SH,23.871091,13.072937,0.248103,0.305582,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,2025-12-25,600519.SH,29.802281,21.099022,0.100529,0.168413,1,0,0,0,1,0,0,1,0,0
215,2025-12-26,600519.SH,27.114108,19.195885,0.114628,0.129965,0,0,0,0,1,0,0,1,0,0
216,2025-12-29,600519.SH,23.587507,25.614526,0.110894,0.099665,0,1,0,0,0,1,0,0,0,0
217,2025-12-30,600519.SH,19.940759,36.148616,0.143401,0.111726,0,0,0,0,1,0,0,1,0,0
